In [ ]:
from pyspark.sql.functions import *
from pyspark.sql import Window

# Read the input file
df = spark.read.csv("path/to/input/file.csv", header=True)

# Convert the date and time columns to timestamps
df = df.withColumn("STATUS_DATETIME", to_timestamp(concat_ws(" ", df.STATUS_DATE, df.STATUS_TIME), "yyyyMMdd HHmmss"))

# Create a window partitioned by ORDER_ID and sorted by STATUS_DATETIME
window_spec = Window.partitionBy("ORDER_ID").orderBy("STATUS_DATETIME")

# Create a column for the previous status
df = df.withColumn("PREV_STATUS", lag("STATUS", 1).over(window_spec))

# Create columns for the start date and end date
df = df.withColumn("START_DATE", when((df.STATUS == "CREATED") | (df.STATUS == "POOL"), df.STATUS_DATETIME))
df = df.withColumn("END_DATE", when((df.STATUS == "COMPLETED") | (df.STATUS == "CANCELLED"), df.STATUS_DATETIME))

# Create a column for the current status
df = df.withColumn("CURRENT_STATUS", when(df.END_DATE.isNotNull(), df.STATUS).otherwise(df.PREV_STATUS))

# Create a column for the duration
df = df.withColumn("DURATION", when(df.END_DATE.isNotNull(), unix_timestamp(df.END_DATE) - unix_timestamp(df.START_DATE)).otherwise(0))

# Select the desired columns and write the output to a file
output_df = df.select("ORDER_ID", "CURRENT_STATUS", "START_DATE", "END_DATE", "DURATION")
output_df.write.csv("path/to/output/file.csv", header=True)


In [ ]:
from pyspark.sql.functions import *
from pyspark.sql import Window

# Read the input file
df = spark.read.csv("path/to/input/file.csv", header=True)

# Convert the date and time columns to timestamps
df = df.withColumn("STATUS_DATETIME", to_timestamp(concat_ws(" ", df.STATUS_DATE, df.STATUS_TIME), "yyyyMMdd HHmmss"))

# Create a window partitioned by ORDER_ID and sorted by STATUS_DATETIME
window_spec = Window.partitionBy("ORDER_ID").orderBy("STATUS_DATETIME")

# Create a column for the previous status
df = df.withColumn("PREV_STATUS", lag("STATUS", 1).over(window_spec))

# Create columns for the start date and end date
df = df.withColumn("START_DATE", when((df.STATUS == "CREATED") | (df.STATUS == "POOL"), df.STATUS_DATETIME))
df = df.withColumn("END_DATE", when((df.STATUS == "COMPLETED") | (df.STATUS == "CANCELLED"), df.STATUS_DATETIME))

# Create a column for the current status
df = df.withColumn("CURRENT_STATUS", when(df.END_DATE.isNotNull(), df.STATUS).otherwise(df.PREV_STATUS))

# Create a column for the duration
df = df.withColumn("DURATION", when(df.END_DATE.isNotNull(), unix_timestamp(df.END_DATE) - unix_timestamp(df.START_DATE)).otherwise(0))

# Select the desired columns and write the output to a file
output_df = df.select("ORDER_ID", "CURRENT_STATUS", "START_DATE", "END_DATE", "DURATION")
output_df.write.mode("overwrite").option("header", "true").csv("path/to/output/file.csv")


In [ ]:
from pyspark.sql.functions import when, min, max, col
from pyspark.sql.types import DoubleType
from pyspark.sql import SparkSession

# Create a SparkSession
spark = SparkSession.builder.appName("OrderStatus").getOrCreate()

# Load the dataset into a Spark dataframe
df = spark.read.csv("https://raw.githubusercontent.com/GTworx/datasets/main/orderStatusData.csv", header=True)

# Convert the time columns to timestamps
df = df.withColumn("EVENT_TIMESTAMP", df["EVENT_TIMESTAMP"].cast("timestamp"))
df = df.withColumn("EVENT_TIME", df["EVENT_TIME"].cast("timestamp"))

# Pivot the dataframe to get the start and end times for each status
pivot = df.groupBy("ORDER_ID", "SUBSCRIBER_ID").pivot("STATUS").agg(min("EVENT_TIMESTAMP"), max("EVENT_TIME"))

# Calculate the start and end times based on the CREATED and POOL statuses
start_date = when(col("CREATED") < col("POOL"), col("CREATED")).otherwise(col("POOL"))
end_date = when(col("COMPLETED").isNotNull(), col("COMPLETED")).otherwise(col("CANCELLED"))

# Calculate the duration in seconds
duration = (end_date.cast("long") - start_date.cast("long")).cast(DoubleType())/60

# Create the result dataframe
result = pivot.select(
    col("ORDER_ID"),
    col("SUBSCRIBER_ID"),
    col("CREATED").alias("START_DATE"),
    end_date.alias("END_DATE"),
    when(duration > 0, duration).otherwise(0).alias("DURATION"),
    when(end_date.isNotNull(), "COMPLETED").otherwise(col("POOL")).alias("STATUS")
).orderBy("ORDER_ID")

# Show the result dataframe
result.show()


In [ ]:
from pyspark.sql.functions import when, min, max, to_timestamp, col

# Assuming the input DataFrame is called "orders_df" and has the columns:
# ORDER_ID, SUBSCRIBER_ID, STATUS, START_DATE, END_DATE, DURATION

# Convert the START_DATE and END_DATE columns to timestamps
orders_df = orders_df.withColumn("START_DATE", to_timestamp(col("START_DATE")))
orders_df = orders_df.withColumn("END_DATE", to_timestamp(col("END_DATE")))

# Create a new column for the minimum start date of CREATED and POOL statuses
min_start_date = when(col("STATUS").isin(["CREATED", "POOL"]), col("START_DATE")).otherwise(None)
orders_df = orders_df.withColumn("MIN_START_DATE", min_start_date.over(Window.partitionBy("ORDER_ID")))

# Create a new column for the maximum end date of COMPLETED and CANCELLED statuses
max_end_date = when(col("STATUS").isin(["COMPLETED", "CANCELLED"]), col("END_DATE")).otherwise(None)
orders_df = orders_df.withColumn("MAX_END_DATE", max_end_date.over(Window.partitionBy("ORDER_ID")))

# Calculate the duration as the difference between the minimum start date and maximum end date
orders_df = orders_df.withColumn("DURATION", (col("MAX_END_DATE").cast("long") - col("MIN_START_DATE").cast("long")) / 3600.0)

# Select only the necessary columns and drop duplicates
result_df = orders_df.select("ORDER_ID", "SUBSCRIBER_ID", "STATUS", "MIN_START_DATE", "MAX_END_DATE", "DURATION") \
                     .dropDuplicates(["ORDER_ID"])


In [ ]:
-- Load sales orders data into a DataFrame
CREATE OR REPLACE TEMPORARY VIEW sales_orders AS
SELECT *
FROM <sales_orders_table>

-- Set start date and end date based on order status
SELECT
  ORDER_ID,
  SUBSCRIBER_ID,
  STATUS,
  CASE
    WHEN STATUS = 'CREATED' THEN LEAST(CREATED_DATE, POOL_DATE)
    ELSE NULL
  END AS START_DATE,
  CASE
    WHEN STATUS IN ('COMPLETED', 'CANCELLED') THEN GREATEST(CANCELLED_DATE, COMPLETED_DATE)
    ELSE NULL
  END AS END_DATE,
  0 AS DURATION
FROM sales_orders

UNION ALL

SELECT
  ORDER_ID,
  SUBSCRIBER_ID,
  STATUS,
  CASE
    WHEN STATUS = 'CREATED' THEN LEAST(CREATED_DATE, POOL_DATE)
    ELSE NULL
  END AS START_DATE,
  CASE
    WHEN STATUS IN ('COMPLETED', 'CANCELLED') THEN GREATEST(CANCELLED_DATE, COMPLETED_DATE)
    ELSE NULL
  END AS END_DATE,
  DATEDIFF(SECOND, LEAST(CREATED_DATE, POOL_DATE), GREATEST(CANCELLED_DATE, COMPLETED_DATE)) AS DURATION
FROM sales_orders
WHERE STATUS IN ('COMPLETED', 'CANCELLED')

-- Group by ORDER_ID to get max duration and latest end date for each order
GROUP BY ORDER_ID
SELECT
  ORDER_ID,
  MAX(DURATION) AS DURATION,
  MAX(END_DATE) AS END_DATE
FROM (
  -- Combine the two sets of data to get all orders, including those that are not completed or cancelled
  SELECT * FROM <first_set_of_data>
  UNION ALL
  SELECT * FROM <second_set_of_data>
)
GROUP BY ORDER_ID

-- Join with the original DataFrame to get the current status
SELECT
  sales_orders.ORDER_ID,
  sales_orders.SUBSCRIBER_ID,
  latest_status.STATUS,
  latest_status.START_DATE,
  latest_status.END_DATE,
  latest_status.DURATION
FROM sales_orders
JOIN (
  SELECT
    ORDER_ID,
    STATUS,
    START_DATE,
    END_DATE,
    DURATION
  FROM (
    -- Rank the rows within each ORDER_ID by END_DATE in descending order
    SELECT
      *,
      RANK() OVER (PARTITION BY ORDER_ID ORDER BY END_DATE DESC) AS row_rank
    FROM <grouped_data>
  ) ranked_data
  WHERE row_rank = 1 -- Get the latest row for each ORDER_ID
) latest_status
ON sales_orders.ORDER_ID = latest_status.ORDER_ID


In [ ]:
SELECT *
FROM <sales_orders_table>

-- Set start date and end date based on order status
SELECT
  ORDER_ID,
  SUBSCRIBER_ID,
  STATUS,
  CASE
    WHEN STATUS = 'CREATED' THEN LEAST(CREATED_DATE, POOL_DATE)
    ELSE NULL
  END AS START_DATE,
  CASE
    WHEN STATUS IN ('COMPLETED', 'CANCELLED') THEN GREATEST(CANCELLED_DATE, COMPLETED_DATE)
    ELSE NULL
  END AS END_DATE,
  0 AS DURATION
FROM sales_orders

UNION ALL

SELECT
  ORDER_ID,
  SUBSCRIBER_ID,
  STATUS,
  CASE
    WHEN STATUS = 'CREATED' THEN LEAST(CREATED_DATE, POOL_DATE)
    ELSE NULL
  END AS START_DATE,
  CASE
    WHEN STATUS IN ('COMPLETED', 'CANCELLED') THEN GREATEST(CANCELLED_DATE, COMPLETED_DATE)
    ELSE NULL
  END AS END_DATE,
  DATEDIFF(SECOND, LEAST(CREATED_DATE, POOL_DATE), GREATEST(CANCELLED_DATE, COMPLETED_DATE)) AS DURATION
FROM sales_orders
WHERE STATUS IN ('COMPLETED', 'CANCELLED')

-- Group by ORDER_ID to get max duration and latest end date for each order
GROUP BY ORDER_ID
SELECT
  ORDER_ID,
  MAX(DURATION) AS DURATION,
  MAX(END_DATE) AS END_DATE
FROM (
  -- Combine the two sets of data to get all orders, including those that are not completed or cancelled
  SELECT * FROM <first_set_of_data>
  UNION ALL
  SELECT * FROM <second_set_of_data>
)
GROUP BY ORDER_ID

-- Join with the original DataFrame to get the current status
SELECT
  sales_orders.ORDER_ID,
  sales_orders.SUBSCRIBER_ID,
  latest_status.STATUS,
  latest_status.START_DATE,
  latest_status.END_DATE,
  latest_status.DURATION
FROM sales_orders
JOIN (
  SELECT
    ORDER_ID,
    STATUS,
    START_DATE,
    END_DATE,
    DURATION
  FROM (
    -- Rank the rows within each ORDER_ID by END_DATE in descending order
    SELECT
      *,
      RANK() OVER (PARTITION BY ORDER_ID ORDER BY END_DATE DESC) AS row_rank
    FROM <grouped_data>
  ) ranked_data
  WHERE row_rank = 1 -- Get the latest row for each ORDER_ID
) latest_status
ON sales_orders.ORDER_ID = latest_status.ORDER_ID